# 摘要:

伴随着基于物理的渲染在实时渲染领域广泛使用，面光源在渲染中的重要性也变得越来越高。对一些特定类型的面光源以及BRDF，存在解析解，然而这些解析解并不能使用在Microfacet BRDF上，目前存在的一些求解方向包括Most Representative Point, Linearly Transform Cosine (LTC), Analytic spherical harmonic coefficients for polygonal area lights等。其中LTC由于其精确性，较好的性能，多种光源类型的支持成为游戏等实时渲染应用程序的首选方案。
然而要将LTC在移动平台的生产项目中使用仍然存在不少挑战。例如horizon clipping 问题，三角形与平面求交从而得到新的多边形，然后在shader里实现这样的函数会造成大量的代码分支以及寄存器的占用。另外还有移动平台上浮点数计算精度的问题，在移动平台上大量的计算将在FP16上进行，然而从之前的实践中我们得知LTC对计算精度有着很高的要求。我们将对这一系列在实践中将会问题的展开讨论。


# 多边形几何体在Lambert表面上的积分

在游戏渲染中，对于一个材质做光照计算通常会涉及一个漫反射以及一个或者多个高光反射的计算。漫反射的BRDF为半球面的cosine函数。  
我们要求解的多边形面光源在半球面的cosine分布函数下
$$
{\frac 1 \pi} {\int_{P} cos(\overrightarrow{\rm n},\overrightarrow{\rm \omega}) } d\omega
$$

存在解析解为:
$$
E(p_1, ..., p_n) = \frac 1 {2 \pi} \sum_{i=1}^{n} acos(\langle p_i, p_j \rangle) \langle {\frac {p_i \times p_j} {\| {p_i \times p_j} \|} }, {\begin{bmatrix} 0\\ 0\\ 1\\ \end{bmatrix}} \rangle \quad \textrm{其中} j = i + 1, p_i为多边形的第i个顶点
$$

原论文中仅给出公式，这里对这个公式给出具体推导。推导主要是使用了[斯托克斯定理](https://en.wikipedia.org/wiki/Stokes%27_theorem)。
$$
\iint _{S}\nabla \times \mathbf {F} \,\cdot \,d\mathbf {S} = \oint _{\Gamma }\mathbf {F} \,\cdot \,d{\mathbf {\Gamma } }
$$

首先我们希望将等式的的左边转换为公式1等价的形式，因为他们的积分域相同，都是球面。斯托克斯定理描述的是一个向量场的旋度与表面法线的点乘。
因为是在球面上，所以任意点的表面法线即为原点到当前点的向量，即为公式1中的$\omega$. 由于两个向量的点乘就是这两个向量的cosine函数，所以如果我们将一个旋度为(0,0,1)的向量场函数代入到斯托克斯定理左边的式子，我们将得到公式1。
对于一个向量场函数$F=(F_x, F_y, F_z)$，对应的旋度定义如下：
$$
(\nabla \times {F})_x = {\frac {\partial F_{z}}{\partial y}}-{\frac {\partial F_{y}}{\partial z}} \\
(\nabla \times {F})_y = {\frac {\partial F_{z}}{\partial x}}-{\frac {\partial F_{y}}{\partial z}} \\
(\nabla \times {F})_z = {\frac {\partial F_{z}}{\partial x}}-{\frac {\partial F_{y}}{\partial y}} 
$$

我们希望得到一个在半球面上旋度恒为(0,0,1)的向量场函数，一个简单的例子为$F(x,y,z)=(0,x,0)$

斯托克斯等式的右边为一个边界积分项，其中对多边形的积分项等于对各个边界的积分项求和。直接求解这样的方程依然不然方便，我们希望可以像球面坐标那样转换一个参数化形式来简化问题，这时候的一个选择是[Slerp函数](https://en.wikipedia.org/wiki/Slerp)，通过这个函数我们可以将球面上两个点的边界积分项转换成对单变量t的积分。
$$
Slerp(p_{i},p_{j},t)={\frac {\sin {((1-t)\Omega })}{\sin \Omega }}p_{i}+{\frac {\sin(t \Omega )}{\sin \Omega }}p_{j} \\
\frac {\partial Slerp(p_{i},p_{j},t)} {\partial t} = - \Omega {\frac {cos((1-t) \Omega)} {sin(\Omega)}} p_i + \Omega {\frac {cos(t \Omega)} {sin(\Omega)}} p_j
$$
其中$\Omega$是$p_0$与$p_1$的夹角。

将$Slerp$与$F(x,y,z)$代入公式3的右边得到：
$$
\oint _{\Gamma }\mathbf {F} \,\cdot \,d{\mathbf {\Gamma } } = \int_{0}^{1} F(Slerp(p_{i},p_{j},t)) \frac {\partial Slerp(p_{i},p_{j},t)} {\partial t} d{t}
 = \frac 1 {2\pi} \sum_i \frac {\Omega_i} {sin(\Omega_i)} (p_{i,x} p_{j,y} - p_{i,y} p_{j,x})
$$

这样就得到了公式1的等价形式，从而完成证明。

对上面给出的解析式求解的结果就是光照计算中的diffuse项，那么剩下的问题就是求解高光反射的计算了。

# 使用Linearly Transformed Cosine来对Microfacet BRDF进行拟合

原论文中表示可以对半球面的cosine函数进行下面这样一个矩阵变换，以此来达到对Microfacet BRDF的拟合。

$$
\begin{bmatrix}
m00 & 0 & m02\\
0 & m11 & 0\\
m20 & 0 & m22
\end{bmatrix}
$$

下面是对这5个变量得直观展示以及其几何意义:  

m00为缩放x轴:  
m02为在ZX平面上旋转:  
m11为缩放y轴:  
m20为偏移度:  
m22为缩放z轴:  

In [ ]:
import matplotlib
from matplotlib import cm, colors
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
# import jax.numpy as np
import scipy.optimize as optimize

In [ ]:
coolwarm_cmap = matplotlib.cm.get_cmap('coolwarm')
coolwarm_rgb = []
norm = matplotlib.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = matplotlib.colors.colorConverter.to_rgb(coolwarm_cmap(norm(i)))
       coolwarm_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = np.array(cmap(k*h)[:3])*255
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

coolwarm_plotly = matplotlib_to_plotly(coolwarm_cmap, 255)

In [ ]:
def spherical_dir(theta, phi):
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return x, y, z

def spherical_coord(x, y, z):
    norm = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/norm)
    phi = np.arctan2(y, x)
    return theta, phi

def meshgrid_spherical_coord(numsamples, hemisphere=False):
    theta = np.linspace(0, np.pi, num=numsamples)
    if hemisphere:
        theta = np.linspace(0, np.pi/2, num=numsamples)
    phi = np.linspace(0, 2*np.pi, num=numsamples*2)
    theta, phi = np.meshgrid(theta, phi)
    return theta, phi

def D(Wi):
    return np.maximum(Wi[2], 0) / np.pi

def D_ltc(Wi, transfo):
    x = Wi[0]
    y = Wi[1]
    z = Wi[2]
    
    inv_transfo = np.linalg.inv(transfo)
    
    x_orig = x * inv_transfo[0][0] + y * inv_transfo[0][1] + z * inv_transfo[0][2]
    y_orig = x * inv_transfo[1][0] + y * inv_transfo[1][1] + z * inv_transfo[1][2]
    z_orig = x * inv_transfo[2][0] + y * inv_transfo[2][1] + z * inv_transfo[2][2]

    l = np.sqrt(x_orig**2 + y_orig**2 + z_orig**2)

    # vector normalization
    x_orig /= l
    y_orig /= l
    z_orig /= l

    # evaluate spherical function
    wi_orig = np.array([x_orig, y_orig, z_orig])
    vals = D(wi_orig)
    
    # apply change of variable
    jacobian = np.linalg.det(inv_transfo) / (l*l*l);
    vals *= jacobian
    
    return vals

def plot_ltc_func(transfo, add_trace, scale_by_value=False):
    numsamples = 64
    theta_ = np.linspace(0, np.pi, num=numsamples)
    phi_ = np.linspace(0, 2*np.pi, num=numsamples*2)
    theta, phi = np.meshgrid(theta_, phi_)

    x, y, z = spherical_dir(theta, phi)
    
    vals = D_ltc(np.array([x, y, z]), transfo)

    # normalize the value for better visualization
    vals /= np.max(vals)

    if (scale_by_value):
        add_trace(go.Surface(x=x*vals, y=y*vals, z=z*vals,
                             surfacecolor=vals,
                             colorscale=coolwarm_plotly,
                             showscale=False))
    else:
        add_trace(go.Surface(x=x, y=y, z=z,
                             surfacecolor=vals,
                             colorscale=coolwarm_plotly,
                             showscale=False))

    # ipv.plot_mesh(x, z, y, wireframe=False, color=cm.coolwarm(vals))
    # ipv.plot_mesh(x*vals, z*vals, y*vals, wireframe=False, color=cm.coolwarm(vals))

    # plot lines in order to visualize the geometry transform
    sample_points = np.array([[0, 0, 2], [-1, -1, 2], [-1, 1, 2], [1, 1, 2], [1, -1, 2]])
    for p in sample_points:
        p = np.dot(transfo, p)
        p /= np.linalg.norm(p)
        p *= 1.5
        add_trace(go.Scatter3d(x=[0, p[0]], y=[0, p[1]], z=[0, p[2]],
                               mode='lines',
                               showlegend=False,
                               line=dict(color='black')))

    # axis_dic = dict(backgroundcolor="rgba(0, 0, 0, 0)",
    #                 visible=False)
    # fig.update_layout(scene=dict(xaxis=axis_dic,
    #                              yaxis=axis_dic,
    #                              zaxis=axis_dic))
    # return fig


def setup_ipv_viev():
    foo = 1
    # ipv.xyzlim(-2, 2)
    # ipv.pylab.view(azimuth=30, elevation=30, distance=2)


def plot(a, b, c, d, e, add_trace, scale_by_value=False):
#     transfo = np.identity(3, dtype = float)
#     transfo[0][0] = a
#     transfo[0][2] = b
#     transfo[1][1] = c
#     transfo[2][0] = d
#     transfo[2][2] = e
    transfo = np.array([[a, 0, b],
                        [0, c, 0],
                        [d, 0, e]])

    plot_ltc_func(transfo, add_trace, scale_by_value=scale_by_value)
    # setup_ipv_viev()
    # ipv.show()

In [ ]:
fig = make_subplots(rows=2, cols=6,
                    specs=[[{'is_3d': True} for _ in range(6)],[{'is_3d': True} for _ in range(6)]],
                    subplot_titles=['m00=0.8', 'm00=0.4', 'm11=0.8', 'm11=0.4','m22=1.2', 'm22=2.0'],
                    horizontal_spacing = 0.01, vertical_spacing = 0.05)

plot(0.8, 0, 1, 0, 1, lambda c: fig.add_trace(c, row=1, col=1))
plot(0.4, 0, 1, 0, 1, lambda c: fig.add_trace(c, row=1, col=2))
plot(1, 0, 0.8, 0, 1, lambda c: fig.add_trace(c, row=1, col=3))
plot(1, 0, 0.4, 0, 1, lambda c: fig.add_trace(c, row=1, col=4))
plot(1, 0, 1, 0, 1.2, lambda c: fig.add_trace(c, row=1, col=5))
plot(1, 0, 1, 0, 2.0, lambda c: fig.add_trace(c, row=1, col=6))
plot(0.8, 0, 1, 0, 1, lambda c: fig.add_trace(c, row=2, col=1), scale_by_value=True)
plot(0.4, 0, 1, 0, 1, lambda c: fig.add_trace(c, row=2, col=2), scale_by_value=True)
plot(1, 0, 0.8, 0, 1, lambda c: fig.add_trace(c, row=2, col=3), scale_by_value=True)
plot(1, 0, 0.4, 0, 1, lambda c: fig.add_trace(c, row=2, col=4), scale_by_value=True)
plot(1, 0, 1, 0, 1.2, lambda c: fig.add_trace(c, row=2, col=5), scale_by_value=True)
plot(1, 0, 1, 0, 2.0, lambda c: fig.add_trace(c, row=2, col=6), scale_by_value=True)

axis_dict = dict(backgroundcolor="rgba(0, 0, 0, 0)", visible=False)
camera_dict = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.8, y=1.2, z=0.8)
)
fig.update_layout(width=3072, height=1024)
for k, i in enumerate(fig['layout']['annotations']):
    if k < 6:
        i['font'] = dict(size=40)
    else:
        i['font'] = dict(size=0)
for k in range(12):
    scene = "scene"
    if k > 0:
        scene += str(k+1)
    camera = scene + "_camera"
    fig['layout'][scene]['xaxis'] = axis_dict
    fig['layout'][scene]['yaxis'] = axis_dict
    fig['layout'][scene]['zaxis'] = axis_dict
    fig['layout'][camera] = camera_dict

# for k, i in enumerate(fig['layout']['scene']):
#     print(i)
#     i['xaxis']['backgroundcolor'] = "rgba(0, 0, 0, 0)"
fig.show()

# aSlider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=1)
# bSlider = widgets.FloatSlider(min=-1, max=1, step=0.01, value=0)
# cSlider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=1)
# dSlider = widgets.FloatSlider(min=-1, max=1, step=0.01, value=0)
# eSlider = widgets.FloatSlider(min=0.01, max=4, step=0.01, value=1)
# widgets.interact(plot, a=aSlider, b=bSlider, c=cSlider, d=dSlider, e=eSlider)
# ipv.show()

In [ ]:
fig = make_subplots(rows=2, cols=6,
                    specs=[[{'is_3d': True} for _ in range(6)], [{'is_3d': True} for _ in range(6)]],
                    subplot_titles=['m02=0.1', 'm02=0.5', 'm02=0.9', 'm20=0.1','m20=0.5', 'm20=0.9'],
                   horizontal_spacing = 0.01, vertical_spacing = 0.05)

plot(1, 0.1, 1, 0, 1, lambda c: fig.add_trace(c, row=1, col=1))
plot(1, 0.5, 1, 0, 1, lambda c: fig.add_trace(c, row=1, col=2))
plot(1, 0.9, 1, 0, 1, lambda c: fig.add_trace(c, row=1, col=3))
plot(1, 0, 1, 0.1, 1, lambda c: fig.add_trace(c, row=1, col=4))
plot(1, 0, 1, 0.5, 1, lambda c: fig.add_trace(c, row=1, col=5))
plot(1, 0, 1, 0.9, 1, lambda c: fig.add_trace(c, row=1, col=6))
plot(1, 0.1, 1, 0, 1, lambda c: fig.add_trace(c, row=2, col=1), scale_by_value=True)
plot(1, 0.5, 1, 0, 1, lambda c: fig.add_trace(c, row=2, col=2), scale_by_value=True)
plot(1, 0.9, 1, 0, 1, lambda c: fig.add_trace(c, row=2, col=3), scale_by_value=True)
plot(1, 0, 1, 0.1, 1, lambda c: fig.add_trace(c, row=2, col=4), scale_by_value=True)
plot(1, 0, 1, 0.5, 1, lambda c: fig.add_trace(c, row=2, col=5), scale_by_value=True)
plot(1, 0, 1, 0.9, 1, lambda c: fig.add_trace(c, row=2, col=6), scale_by_value=True)

axis_dict = dict(backgroundcolor="rgba(0, 0, 0, 0)", visible=False)
camera_dict = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.8, y=1.2, z=0.8)
)
camera2_dict = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=2, z=0)
)

fig.update_layout(width=3072, height=1024)
for k, i in enumerate(fig['layout']['annotations']):
    if k < 6:
        i['font'] = dict(size=40)
    else:
        i['font'] = dict(size=0)
for k in range(12):
    scene = "scene"
    if k > 0:
        scene += str(k+1)
    camera = scene + "_camera"
    fig['layout'][scene]['xaxis'] = axis_dict
    fig['layout'][scene]['yaxis'] = axis_dict
    fig['layout'][scene]['zaxis'] = axis_dict
    if k < 6:
        fig['layout'][camera] = camera_dict
    else:
        fig['layout'][camera] = camera2_dict

fig.show()

然而要直接求解这个5维的优化问题还是很困难得，我们要想办法降低问题的复杂性。  
首先我们先将问题简化，使用Phong模型来替游戏中使用的GGX模型，在Phong模型的情况下，可以将cosine lobe通过缩放与旋转矩阵来达到对phong的拟合，其中旋转矩阵的Z轴为反射向量，由于Phong模型是各项同性的，所以缩放矩阵的m00与m11相等，问题简化为单变量的优化问题，就很容易解决了。  
下面我们来分析一个具体的例子，当视线方向与法线的夹角为30度，且物体表面的粗糙度为0.6时，我们通过求解单变量的L2 norm优化问题，得到缩放系数为0.443，LTC矩阵为
$$
\begin{bmatrix}
0.384 & 0 & 0.5\\
0 & 0.443 & 0\\
-0.2215 & 0 & 0.867
\end{bmatrix}
$$
下面为对比结果:

Phong BRDF                          | LTC
------------------------------------|-------------------------------------
![](images/phong_roughness-0.6_theta-0.523.png)  | ![](images/phong_approx.png) 


In [ ]:
def spherical_integral(integrand, num_samples=128, hemisphere=False):
    theta_max = np.pi
    if hemisphere:
        theta_max = np.pi * 0.5

    phi_max = np.pi * 2

    theta = np.linspace(0, theta_max, num_samples)
    phi = np.linspace(0, phi_max, num_samples)
    theta, phi = np.meshgrid(theta, phi)
    vec = np.array(spherical_dir(theta, phi))

    v = integrand(vec)
    dim = len(v.shape)
    integral = np.sum(v * np.sqrt(1 - vec[2] ** 2), axis=(dim-2, dim-1)) * theta_max * phi_max / num_samples / num_samples
    return integral

In [ ]:
# Has cosine builtin
def phong_cbrdf(theta, roughness, wi):
    wo = np.array([-np.sin(theta), 0, np.cos(theta)])
    r = wo
    r = np.array([-r[0], r[1], r[2]])
    
    cosThetaH = np.maximum((wi[0]*r[0] + wi[1]*r[1] + wi[2]*r[2]), 0)
    alpha = roughness * roughness
    a2 = alpha * alpha
    power = 2/a2 - 2
    norm = 1.0/np.pi/a2
    return norm * np.power(cosThetaH, power) * np.maximum(wi[2], 0)

def Gvis_GGX(NdotV, NdotL, alpha):
    a2 = alpha*alpha
    G_V = NdotV + np.sqrt( (NdotV - NdotV * a2) * NdotV + a2 )
    G_L = NdotL + np.sqrt( (NdotL - NdotL * a2) * NdotL + a2 )
    return 1.0/( G_V * G_L )

def ggx_cbrdf(theta, roughness, wi):
    wo = np.array([-np.sin(theta), 0, np.cos(theta)])
    wh = np.array([wi[0] + wo[0], wi[1] + wo[1], wi[2] + wo[2]])
    wh /= np.sqrt(wh[0]**2 + wh[1]**2 + wh[2]**2)

    NoH = np.maximum(wh[2], 0)
    NoV = np.maximum(wo[2], 0)
    NoL = np.maximum(wi[2], 0)

    alpha = roughness * roughness
    a2 = alpha * alpha
    d = ((NoH * a2 - NoH) * NoH + 1)
    NDF = a2 / (np.pi * d * d)
    G = Gvis_GGX(NoV, NoL, alpha)
    return NDF * G *  NoL

def ggx_cbrdf_dominant_dir(theta, roughness, wi):
    wo = np.array([-np.sin(theta), 0, np.cos(theta)])
    wh = np.array([wi[0] + wo[0], wi[1] + wo[1], wi[2] + wo[2]])
    wh /= np.sqrt(wh[0]**2 + wh[1]**2 + wh[2]**2)

    NoH = np.maximum(wh[2], 0)
    NoV = np.maximum(wo[2], 0)
    NoL = np.maximum(wi[2], 0)

    alpha = roughness * roughness
    a2 = alpha * alpha
    d = ((NoH * a2 - NoH) * NoH + 1)
    NDF = a2 / (np.pi * d * d)
    G = Gvis_GGX(NoV, NoL, alpha)
    ret = NDF * G *  NoL
    ind = np.unravel_index(np.argmax(ret, axis=None), ret.shape)
    print("max index: ", ind)
    direction = np.array([wi[0][ind], wi[1][ind], wi[2][ind]])
    print("max value: ", ret[ind], np.max(ret))
    return direction

def getSpecularDominantDir(N, R, alpha):
    a2 = alpha * alpha;
    return R * (1-a2) + N*a2;

    smoothness = 1 - alpha
    factor = smoothness * (np.sqrt(smoothness) + alpha)
    return N * (1 - factor) + R * factor


theta_view = 60/180*np.pi
roughness = np.sqrt(0.36)

theta, phi = meshgrid_spherical_coord(256, hemisphere=True)
x, y, z = spherical_dir(theta, phi)
wi = np.array([x, y, z])

phong_norm = spherical_integral(lambda wi: phong_cbrdf(theta_view, roughness, wi), num_samples=1024, hemisphere=True)
print("Phong normalization: ", phong_norm)
vals = phong_cbrdf(theta_view, roughness, wi)

ggx_norm = spherical_integral(lambda wi: ggx_cbrdf(theta_view, roughness, wi), num_samples=1024, hemisphere=True)

def ggx_cbrdf_average_term(wi):
    v = ggx_cbrdf(theta_view, roughness, wi)
    return v

ggx_dir_x = spherical_integral(lambda wi: ggx_cbrdf(theta_view, roughness, wi) * wi[0],
                             num_samples=1024,
                             hemisphere=True)
ggx_dir_z = spherical_integral(lambda wi: ggx_cbrdf(theta_view, roughness, wi) * wi[2],
                             num_samples=1024,
                             hemisphere=True)
ggx_dir = np.array([ggx_dir_x, 0, ggx_dir_z])

ggx_dir = ggx_cbrdf_dominant_dir(theta_view, roughness, wi)

# ggx_dir_y = spherical_integral(lambda wi: ggx_cbrdf_average_term(wi) * wi[1],
#                              num_samples=1024,
#                              hemisphere=True)
# ggx_dir_z = spherical_integral(lambda wi: ggx_cbrdf_average_term(wi) * wi[2],
#                              num_samples=1024,
#                              hemisphere=True)
# ggx_dir = np.array([ggx_dir_x, ggx_dir_y, ggx_dir_z])
reflect_dir = np.array([np.sin(theta_view), 0, np.cos(theta_view)])
# ggx_dir = getSpecularDominantDir(np.array([0,0,1]), reflect_dir, roughness*roughness)
# ggx_dir /= np.linalg.norm(ggx_dir)
print("ggx dir ", ggx_dir)
# ggx_dir = np.array([0.625941455, 0, 0.779869974])

print("GGX normalization: ", ggx_norm)
ggx_vals = ggx_cbrdf(theta_view, roughness, wi)
phong_vals = phong_cbrdf(theta_view, roughness, wi)


ggx_cbrdf_approx = ggx_approx(0.79849551,  0.38944701, -0.14630333, ggx_norm, ggx_dir)
ggx_norm_approx = spherical_integral(lambda wi: ggx_cbrdf_approx(wi), num_samples=1024, hemisphere=False)
print("GGX LTC normalization: ", ggx_norm_approx)
# ggx_cbrdf_approx = ggx_approx(1.02277543,  0.621345  , -0.1516238, ggx_norm/ggx_norm_approx)
# print("GGX LTC normalization: ", spherical_integral(lambda wi: ggx_cbrdf_approx(wi), num_samples=1024, hemisphere=False))
ggx_vals_approx = ggx_cbrdf_approx(wi)

# normalize the value for better visualization
# vals /= np.max(vals)

max_val = np.max(ggx_vals)

# max_val = np.maximum(max_val, np.max(ggx_vals_approx))


fig = go.Figure(data=go.Surface(x=x*ggx_vals, y=y*ggx_vals, z=z*ggx_vals,
                                colorscale=coolwarm_plotly,
                                surfacecolor=ggx_vals,
                                opacity=0.6))

fig.add_trace(go.Surface(x=x*ggx_vals_approx, y=y*ggx_vals_approx, z=z*ggx_vals_approx,
                                colorscale=coolwarm_plotly,
                                surfacecolor=ggx_vals_approx,
                                opacity=0.6))

fig.add_trace(go.Scatter3d(x=[0, np.sin(theta_view) * 6], y=[0, 0], z=[0, np.cos(theta_view) * 6],
                           mode='lines',
                           showlegend=False,
                           line=dict(color='black')))
fig.add_trace(go.Scatter3d(x=[0, ggx_dir[0]*6], y=[0, ggx_dir[1]*6], z=[0, ggx_dir[2]*6],
                           mode='lines',
                           showlegend=False,
                           line=dict(color='red')))

fig.add_trace(go.Scatter3d())
fig.update_layout(scene=dict(xaxis=dict(range=[-max_val,max_val]),
                             yaxis=dict(range=[-max_val,max_val]),
                             zaxis=dict(range=[-max_val,max_val]),
                             aspectmode='manual',
                             aspectratio=dict(x=1, y=1, z=1)))
fig.show()

In [ ]:
def phong_transfo_matrix(a):
    scale_mat = np.identity(3)
    scale_mat[0][0] = a
    scale_mat[1][1] = a
#     scale_mat = np.array([[a, 0, 0],
#                           [0, a, 0],
#                           [0, 0, 1]])

    rotate_mat = np.identity(3)
    reflect_vec = np.array([np.sin(theta_view), 0, np.cos(theta_view)])
    Z = reflect_vec
    Y = np.array([0, 1, 0])
    X = np.cross(Y, Z)
    X /= np.linalg.norm(X)
#     rotate_mat[0] = X
#     rotate_mat[1] = Y
#     rotate_mat[2] = Z
    rotate_mat = np.array([X, Y, Z])
    rotate_mat = np.transpose(rotate_mat)

    transfo = rotate_mat @ scale_mat
    return transfo

def phong_approx(a, norm):
    def f(wi):
        transfo = phong_transfo_matrix(a)
        return D_ltc(wi, transfo) * norm
    return f

def ggx_transfo_matrix(m00, m11, m02, z_axis):
    scale_mat = np.identity(3)
    scale_mat = np.array([[m00, 0, m02],
                          [0, m11, 0],
                          [0, 0, 1]])

    rotate_mat = np.identity(3)
    reflect_vec = np.array([np.sin(theta_view), 0, np.cos(theta_view)])
    reflect_vec = z_axis
    Z = reflect_vec
    Y = np.array([0, 1, 0])
    X = np.cross(Y, Z)
    X /= np.linalg.norm(X)
#     rotate_mat[0] = X
#     rotate_mat[1] = Y
#     rotate_mat[2] = Z
    rotate_mat = np.array([X, Y, Z])
    rotate_mat = np.transpose(rotate_mat)

    transfo = rotate_mat @ scale_mat
    return transfo

def ggx_approx(m00, m11, m02, norm, z_axis):
    def f(wi):
        transfo = ggx_transfo_matrix(m00, m11, m02, z_axis)
        return D_ltc(wi, transfo) * norm
    return f

In [ ]:
ggx_transfo_matrix(0.63028784,  0.41804157, -0.07809715, ggx_dir)

In [ ]:
fig = make_subplots(rows=1, cols=5,
                    specs=[[{'is_3d': True} for _ in range(5)]],
                    subplot_titles=['Cosine函数', '对m00,m11进行缩放', '对m02进行偏移', '以BRDF主要方向为Z轴', 'GGX BRDF'],
                    horizontal_spacing = 0.01, vertical_spacing = 0.05)

fig.add_trace(go.Surface(x=x, y=y, z=z, surfacecolor=z, colorscale=coolwarm_plotly, showscale=False), row=1, col=1)

ggx_cbrdf_approx = ggx_approx(0.63028784,  0.41804157, 0, ggx_norm, np.array([0.0,0.0,1.0]))
ggx_vals_approx = ggx_cbrdf_approx(wi)
fig.add_trace(go.Surface(x=x, y=y, z=z,
                         surfacecolor=ggx_vals_approx,
                         colorscale=coolwarm_plotly,
                         showscale=False,
                         opacity=1.0),
              row=1, col=2)

ggx_cbrdf_approx = ggx_approx(0.63028784,  0.41804157, -0.07809715, ggx_norm, np.array([0.0,0.0,1.0]))
ggx_vals_approx = ggx_cbrdf_approx(wi)
fig.add_trace(go.Surface(x=x, y=y, z=z,
                         surfacecolor=ggx_vals_approx,
                         colorscale=coolwarm_plotly,
                         showscale=False,
                         opacity=1.0),
              row=1, col=3)

ggx_cbrdf_approx = ggx_approx(0.63028784,  0.41804157, -0.07809715, ggx_norm, ggx_dir)
ggx_vals_approx = ggx_cbrdf_approx(wi)
fig.add_trace(go.Surface(x=x, y=y, z=z,
                         surfacecolor=ggx_vals_approx,
                         colorscale=coolwarm_plotly,
                         showscale=False,
                         opacity=1.0),
              row=1, col=4)

ggx_vals = ggx_cbrdf(theta_view, roughness, wi)
fig.add_trace(go.Surface(x=x, y=y, z=z, surfacecolor=ggx_vals, showscale=False, colorscale=coolwarm_plotly),
              row=1, col=5)

fig.update_layout(width=2560, height=512)
for k, i in enumerate(fig['layout']['annotations']):
    i['font'] = dict(size=40)

for k in range(5):
    scene = "scene"
    if k > 0:
        scene += str(k+1)
    fig['layout'][scene]['xaxis'] = axis_dict
    fig['layout'][scene]['yaxis'] = axis_dict
    fig['layout'][scene]['zaxis'] = axis_dict

# max_val = np.max(vals)
# fig.update_layout(scene=dict(xaxis=dict(range=[-max_val,max_val]),
#                              yaxis=dict(range=[-max_val,max_val]),
#                              zaxis=dict(range=[-max_val,max_val]),
#                              aspectmode='manual',
#                              aspectratio=dict(x=1, y=1, z=1)))
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=4,
                    specs=[[{'is_3d': True} for _ in range(4)]],
                    subplot_titles=['Cosine函数', '对m00,m11进行缩放', '以反射向量为Z轴', 'GGX BRDF'],
                    horizontal_spacing = 0.01, vertical_spacing = 0.05)

fig.add_trace(go.Surface(x=x, y=y, z=z, surfacecolor=z, colorscale=coolwarm_plotly, showscale=False), row=1, col=1)

theta_view = 0/180*np.pi
ltc_rotated = phong_approx(0.38, phong_norm)
vals = ltc_rotated(wi)

# ltc_norm = spherical_integral(lambda wi: ltc_rotated(wi), num_samples=4096, hemisphere=True)
# print("ltc normalization: ", ltc_norm)

# normalize the value for better visualization
# vals /= np.max(vals)
fig.add_trace(go.Surface(x=x, y=y, z=z,
                         surfacecolor=vals,
                         colorscale=coolwarm_plotly,
                         showscale=False,
                         opacity=1.0),
              row=1, col=2)

theta_view = 30/180*np.pi
vals_approx = phong_approx(0.38, phong_norm)(wi)
fig.add_trace(go.Surface(x=x, y=y, z=z,
                         surfacecolor=vals_approx,
                         colorscale=coolwarm_plotly,
                         showscale=False,
                         opacity=1.0),
              row=1, col=3)

r_vals = phong_cbrdf(theta_view, roughness, wi)
fig.add_trace(go.Surface(x=x, y=y, z=z, surfacecolor=r_vals, showscale=False, colorscale=coolwarm_plotly),
              row=1, col=4)

fig.update_layout(width=2048, height=512)
for k, i in enumerate(fig['layout']['annotations']):
    i['font'] = dict(size=40)

for k in range(4):
    scene = "scene"
    if k > 0:
        scene += str(k+1)
    fig['layout'][scene]['xaxis'] = axis_dict
    fig['layout'][scene]['yaxis'] = axis_dict
    fig['layout'][scene]['zaxis'] = axis_dict

# max_val = np.max(vals)
# fig.update_layout(scene=dict(xaxis=dict(range=[-max_val,max_val]),
#                              yaxis=dict(range=[-max_val,max_val]),
#                              zaxis=dict(range=[-max_val,max_val]),
#                              aspectmode='manual',
#                              aspectratio=dict(x=1, y=1, z=1)))
fig.show()

In [ ]:
def obj_fn(p):
    m00, m11, m02 = p
    f = ggx_approx(m00, m11, m02, ggx_norm, ggx_dir)
    # norm = spherical_integral(lambda wi: f(wi), num_samples=1024, hemisphere=True)
    # f = ggx_approx(m00, m11, m20, ggx_norm/norm)

    theta, phi = meshgrid_spherical_coord(512, hemisphere=True)
    x, y, z = spherical_dir(theta, phi)
    vals_approx = f(np.array([x, y, z]))
    vals_exact = ggx_cbrdf(theta_view, roughness, np.array([x, y, z]))

    diff = np.abs(vals_exact - vals_approx)**2
    return np.mean(diff)

initial_guess = [0.6301864 ,  0.4180192 , 0]
result = optimize.minimize(obj_fn, initial_guess, method="Nelder-Mead")

print(result)
# result.x = 0.443
# print(ggx_transfo_matrix(result.x))
# f = phong_approx(result.x)
# theta, phi = meshgrid_spherical_coord(64)
# x, y, z = spherical_dir(theta, phi)
# vals = f(np.array([x, y, z]))
# vals_exact = phong_cbrdf(theta_view, roughness, np.array([x, y, z]))
# # normalize the value for better visualization
# vals = vals - vals_exact
# # vals /= np.max(vals)
# ipv.clear()
# ipv.plot_mesh(x*vals, z*vals, y*vals, wireframe=False, color=cm.coolwarm(vals))
# setup_ipv_viev()
# ipv.show()

In [ ]:
import jax


def loss(params):
    a = params
    return obj_fn(a)

def update_parameters_step(params, learning_rate=0.001):
    grad_loss = jax.grad(loss)
    grads = grad_loss(params)
    return params - learning_rate * grads

def optimize_loop(x0, print_loss = False):
    NUM_STEPS = 50000
    for n in range(NUM_STEPS):
        x0 = update_parameters_step(x0)
        if print_loss and n % 10000 == 0:
            print(x0)
    return x0

result = optimize_loop(1.0, print_loss=True)
print(result)

In [ ]:
fig = ipv.figure(lighting=False)

def plot_phong(theta, scale):
    scale_mat = np.identity(3, dtype = float)        
    scale_mat[0][0] = scale
    scale_mat[1][1] = scale
    rotate_mat = np.identity(3, dtype = float)
    rotate_mat[2] = np.array([-np.sin(theta), 0, np.cos(theta)])
    rotate_mat[0] = np.cross(rotate_mat[1], rotate_mat[2])
    transfo = rotate_mat @ scale_mat

    fig.meshes.clear()
    fig.scatters.clear()
    plot_ltc_func(transfo)
    ipv.xyzlim(-2, 2)


thetaSlider = widgets.FloatSlider(min=0, max=np.pi*0.5, step=0.01, value=0)
scaleSlider = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=1)
widgets.interact(plot_phong, theta=thetaSlider, scale=scaleSlider)
ipv.show()

有了上面的经验之后，我们再尝试将Phong模型替换成GGX。下图为Phong与GGX lobe的对比:
![](images/ggx_phong.png)

从上面我们可以观察到，GGX为各项异性且lobe的主要方向与反射向量存在一定夹角。我们继续按照将矩阵分解成旋转与缩放矩阵的思路进行求解，其中旋转矩阵与Phong模型一致。此时通过我们观察到的GGX对比Phong的性质，可以通过m00与m11来对X,Y方向的缩放以及m20对YZ平面上的旋转来达到GGX的近似，具体效果见下图：

最终我们通过将LTC变换矩阵分解成旋转矩阵以及本地变换矩阵（X,Y方向的缩放以及YZ平面上的旋转），将一个5维的优化问题简化成3维的优化问题。

生成的LUT需要存储5个变量，这样需要两次贴图查找操作，我们希望将变量控制在4个以内，这样只需要一次查表操作。让我们回顾一下LTC中关于D的解析式
$$
D(\omega) = D_o({\omega}_o) {\frac {\partial {\omega}_o} {\partial \omega}} 
          = D_o (\frac {M^{-1} \omega} {\| {M^{-1} \omega} \|}) {\frac {| M^{-1} |} {{\| {|M^{-1} \omega} \|}^3}}
$$

其中$M^{-1}$为我们生成的LUT表。通过上的解析式分析我们可以得到，将$\lambda I M^{-1}$替换掉$M^{-1}$，上面表达式依然成立, 其中$I$为单位矩阵。
$$
D(\omega) = D_o (\frac {\lambda I  M^{-1} \omega} {\| \lambda I {M^{-1} \omega} \|}) {\frac {| \lambda I M^{-1} |} {{\| \lambda I {M^{-1} \omega} \|}^3}}  \\
          = D_o (\frac {M^{-1} \omega} {\| {M^{-1} \omega} \|}) {\frac {{| \lambda I|} {| M^{-1} |}} { {{\lambda}^3} {{\| {M^{-1} \omega} \|}^3}} } \\
          = D_o (\frac {M^{-1} \omega} {\| {M^{-1} \omega} \|}) {\frac {{{\lambda}^3} {| M^{-1} |}} { {{\lambda}^3} {{\| {M^{-1} \omega} \|}^3}} }
$$

所以我们可以将LUT矩阵除以m00, m11, m22任意一个来达到将数据压缩成4个的目的。同时我们也可以对M矩阵进行归一化，这里就不给出证明了。  
下图为分别使用m00, m11, m22来进行归一化后的数据展示。

In [ ]:
import ltc_ggx_lut as lut
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


def lut_M(i, j):
    comp = np.zeros((64, 64)) 
    for t in np.arange(64):
        for s in np.arange(64):
            comp[t, s] = lut.ltc_matrix[t, s][i, j]
    return comp

def lut_invM (i, j):
    comp = np.zeros((64, 64)) 
    for t in np.arange(64):
        for s in np.arange(64):
            comp[t, s] = np.linalg.inv(lut.ltc_matrix[t, s])[i, j]
    return comp

m00 = lut_M(0, 0)
m02 = lut_M(0, 2)
m11 = lut_M(1, 1)
m20 = lut_M(2, 0)
m22 = lut_M(2, 2)

m00_invM = lut_invM(0, 0)
m02_invM = lut_invM(0, 2)
m11_invM = lut_invM(1, 1)
m20_invM = lut_invM(2, 0)
m22_invM = lut_invM (2, 2)

这是原始得M矩阵数据

![](images/lut_M.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

def setup_axes(ax):
    view_elev = 45
    ax.view_init(elev=view_elev)
    ax.set_proj_type('ortho')

fig = plt.figure(figsize=plt.figaspect(0.1))

ax = fig.add_subplot(1, 5, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m00,cmap='coolwarm')

ax = fig.add_subplot(1, 5, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02,cmap='coolwarm')

ax = fig.add_subplot(1, 5, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m11, cmap='coolwarm')

ax = fig.add_subplot(1, 5, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m20,cmap='coolwarm')

ax = fig.add_subplot(1, 5, 5, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m22,cmap='coolwarm')

plt.figure(figsize=[32,16])
plt.show()

M通过m00进行归一化
![](images/lut_M_m00_norm.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

fig = plt.figure(figsize=plt.figaspect(0.1))

m02_norm = m02/m00
m11_norm = m11/m00
m20_norm = m20/m00
m22_norm = m22/m00

ax = fig.add_subplot(1, 4, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m11_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m20_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m22_norm,cmap='coolwarm')

plt.show()


M通过对m11进行归一化
![](images/ltc_M_m11_norm.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

fig = plt.figure(figsize=plt.figaspect(0.1))

m00_norm = m00/m11
m02_norm = m02/m11
m20_norm = m20/m11
m22_norm = m22/m11

ax = fig.add_subplot(1, 4, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m00_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m20_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m22_norm,cmap='coolwarm')

plt.show()


M通过对m22进行归一化
![](images/ltc_M_m22_norm.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

fig = plt.figure(figsize=plt.figaspect(0.1))

m00_norm = m00/m22
m02_norm = m02/m22
m11_norm = m11/m22
m20_norm = m20/m22

ax = fig.add_subplot(1, 4, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m00_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m11_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, 1-m20_norm,cmap='coolwarm')

plt.show()


$M^{-1}$通过对m00进行归一化
![](images/ltc_invM_00_norm.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

fig = plt.figure(figsize=plt.figaspect(0.1))

m02_norm = m02_invM/m00_invM
m11_norm = m11_invM/m00_invM
m20_norm = m20_invM/m00_invM
m22_norm = m22_invM/m00_invM

ax = fig.add_subplot(1, 4, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m11_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m20_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m22_norm,cmap='coolwarm')

plt.show()


$M^{-1}$通过对m11进行归一化
![](images/ltc_invM_11_norm.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

fig = plt.figure(figsize=plt.figaspect(0.1))

m00_norm = m00_invM/m11_invM
m02_norm = m02_invM/m11_invM
m20_norm = m20_invM/m11_invM
m22_norm = m22_invM/m11_invM

ax = fig.add_subplot(1, 4, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m00_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m20_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m22_norm,cmap='coolwarm')

plt.show()


$M^{-1}$通过对m22进行归一化
![](images/ltc_invM_22_norm.png)

In [ ]:
x = np.arange(64)
y = 63-np.arange(64)
x, y = np.meshgrid(x, y)

fig = plt.figure(figsize=plt.figaspect(0.1))

m00_norm = m00_invM/m22_invM
m02_norm = m02_invM/m22_invM
m11_norm = m11_invM/m22_invM
m20_norm = m20_invM/m22_invM

ax = fig.add_subplot(1, 4, 1, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m00_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 2, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m02_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 3, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, m11_norm,cmap='coolwarm')

ax = fig.add_subplot(1, 4, 4, projection='3d')
setup_axes(ax)
surf = ax.plot_surface(x, y, 1-m20_norm,cmap='coolwarm')

plt.show()


从上面的结果我们可以知道将$M^{-1}$通过m11进行归一化，将得到最好的结果。

# 简单实现

下面为shader的实现代码，其中:  
N: 法线向量  
V: 视口向量  
P: 当前计算着色点的坐标  
points: 光源的顶点位置  
NoV: N与V的点乘  
linearRoughness: 材质的粗糙度  
twoSided: 光源是否为双面  
```
float evaluateQuadLight(float3 N, float3 V, float3 P, float3 points[4], float NoV, float linearRoughness, bool twoSided)
{
    float3x3 invM = getLtcMatrix(NoV, linearRoughness);

    /* 构建本地坐标系，其中物体法线N为Z轴，视口向量V处于XZ平面 */
    float3x3 basis;
    basis[0] = normalize(V - N * dot(N, V));
    basis[1] = normalize(cross(N, basis[0]));
    basis[2] = N;

    invM = mul(transpose(basis), invM);

    /* 将区域光的各顶点变换到本地坐标系下 */
    float3 L[5];
    L[0] = mul(points[0] - P, invM);
    L[1] = mul(points[1] - P, invM);
    L[2] = mul(points[2] - P, invM);
    L[3] = mul(points[3] - P, invM);
    L[4] = 0;

    /* 对投影到球面上的顶点进行上半球的水平面面裁切(horizon clipping)，可能产生新的多边形 */
    int n = clipQuadToHorizon(L);

    if(n == 0)
        return 0.0;

    // 投影到球面上
    L[0] = normalize(L[0]);
    L[1] = normalize(L[1]);
    L[2] = normalize(L[2]);
    L[3] = normalize(L[3]);
    L[4] = normalize(L[4]);

    /* 使用lambert公式进行积分计算 */
    float sum = integrateEdge(L[0], L[1])
              + integrateEdge(L[1], L[2])
              + integrateEdge(L[2], L[3]);
    if(n >= 4)
        sum += integrateEdge(L[3], L[4]);
    if(n == 5)
        sum += integrateEdge(L[4], L[0]);

    sum *= 1.0 / (2 * UNITY_PI);

    float sum = 0;

    sum = twoSided ? abs(sum) : max(sum, 0.0);

    return sum;
}
```

上面的shader代码给出了一个简单的实现，然后这里还有更多的问题需要解决，首先clipQuadToHorizon这个函数里有大量的代码分支，这对GPU的运行机制非常不友好，另外一个是integrateEdge函数里将会碰到的数值计算精度的问题。

# 水平面裁切

第一章节中提到的多边形在lambert表面的积分是对clamped cosine函数进行求积，所以我们需要将多边形裁切到上半球，shader里实现多边形的裁切会造成大量的代码分支，以下为伪代码示例。

```
int clipQuadToHorizon(inout float3 L[5])
{
    /* Detect clipping config */
    int config = 0;
    if(L[0].z > 0.0) config += 1;
    if(L[1].z > 0.0) config += 2;
    if(L[2].z > 0.0) config += 4;
    if(L[3].z > 0.0) config += 8;

    if(config == 0) {
        // clip all
    } else if(config == 1) { // V1 clip V2 V3 V4
        ...
    } else if(config == 2) { // V2 clip V1 V3 V4
        ...
    ...
    } else if(config == 14) { // V2 V3 V4 clip V1
        ...
    } else if(config == 15) { // V1 V2 V3 V4
        n = 4;
    }
    
    if(n == 3)
        L[3] = L[0];
    if(n == 4)
        L[4] = L[0];

    return n;
}
```

原论文并没有对裁切这部分的实践做介绍，不过Stephen Hill在[Real-Time Area Lighting:a Journey from Research to Production](https://advances.realtimerendering.com/s2016/s2016_ltc_rnd.pdf)中给出了一个近似方案，这里做一下简单的介绍。

我们可以将horizon clipping的求解分解成下面两个部分
polygonal_integration_without_horizon_clipping * horizon_clipping_factor
其中polygonal_integration_without_horizon_clipping可以直接求解公式1得到，我们的目标就变成求解horizon_clipping_factor。这时候的思路是通过更简单的几何代理来近似这个值，最简单的几何代理就是球了，球与平面的horizon clipping存在解析解。于是这时候问题转变成如何选择球的半径与朝向。

如果将公式1中与法线向量(<0,0,1>向量)的点乘去掉，那么我们会得到这个form factor的向量形式，这个向量形式与另外一个单位向量U的点乘结果为当前的form factor在以U为Z轴的cosine球面分布函数上的积分。我们可以将vector form factor的方向做为球的朝向，同时选择form factor相同的球。通过下面的公式求解得到, 详细推导参考[sphere ambient occlusion](https://www.iquilezles.org/www/articles/sphereao/sphereao.htm)
$$
angular\_extent = asin(sqrt(length(F)))
$$

这样我们就得到了这个球的张角$sigma$和球与法线的夹角$omega$，这种情况horizon clipping factor存在解析解如下

$$
\begin{align}
I_{hemisphere}(\omega, \sigma) = \frac 1 \pi \left\{ \begin{array}{cc} 
                \pi cos({\omega}) sin^2({\sigma}) & \omega \in [0,\frac \pi 2 - \sigma] \\
                \pi cos({\omega}) sin^2({\sigma}) + G(\omega, \sigma, \gamma) - H(\omega, \sigma, \gamma) & \omega \in [\frac \pi 2 - \sigma, \frac \pi 2] \\
                G(\omega, \sigma, \gamma) + H(\omega, \sigma, \gamma) & \omega \in [\frac \pi 2,\frac \pi 2 + \sigma] \\
                0 & \omega \in [\frac \pi 2 + \sigma, \pi]
                \end{array} \right.
\end{align}
$$

其中:

$$
\gamma = sin^{-1}(\frac {cos(\sigma)} {sin(\omega)}) \\
G(\omega, \sigma, \gamma) = -2 sin(\omega) cos(\sigma) cos(\gamma) + {\frac \pi 2} - \gamma + sin(\gamma) cos(\gamma) \\
H(\omega, \sigma, \gamma) = cos(\omega) (cos(\gamma) \sqrt {sin^2(\sigma) - cos^2(\gamma)} + sin^2(\sigma) sin^{-1}(\frac {cos(\gamma)} {sin(\sigma)})
$$



这个计算还是过于复杂，我们可以将这个结果保存为贴图，在运行时求$sigma$与$omega$，再通过这两个参数通过查表的方式来求解。

由于使用球来近似平面，无法处理平面的朝向问题，所以我们需要自己来处理平面的朝向，将当前位置带入平面方程就可以得到当前位置处于 平面的正面或者方面，通过对F向量取反即可处理双面光源。修改后的shader代码如下:

```
float evaluateQuadLight(float3 N, float3 V, float3 P, float3 points[4], float4 planeEquation, float NoV, float linearRoughness, bool twoSided)
{
    float3x3 invM = getLtcMatrix(NoV, linearRoughness);

    /* 构建本地坐标系，其中物体法线N为Z轴，视口向量V处于XZ平面 */
    invM = mul(transpose(basis), invM);

    /* 将区域光的各顶点变换到本地坐标系，并投影到球面上 */    
    float3 L[4];
    L[0] = normalize(mul(points[0] - P, invM));
    L[1] = normalize(mul(points[1] - P, invM));
    L[2] = normalize(mul(points[2] - P, invM));
    L[3] = normalize(mul(points[3] - P, invM));

    /* 不考虑horizon clipping, 直接求解form factor的向量形式 */
    float3 F = 0;
    F += integrateEdgeVectorForm(L[0], L[1]);
    F += integrateEdgeVectorForm(L[1], L[2]);
    F += integrateEdgeVectorForm(L[2], L[3]);
    F += integrateEdgeVectorForm(L[3], L[0]);
    F *= M_INV_TWO_PI;

    /* 判断当前着色点是否是在区域光的背面 */
    bool frontface = (dot(planeEquation.xyz, P) + planeEquation.w) > 0;
    if (!twoSided && !frontface)
    {
        return 0;
    }
    else
    {
        F *= frontface ? 1.0 : -1.0;
    }

    /* 根据form factor的向量形式，选择球为几何代码时的参数 */
    float squaredSinSigma = length(F);
    float sinSigma = sqrt(squaredSinSigma);

    float3 normFormFactor = normalize(F);
    float cosOmega = normFormFactor.z;

    /* 根据球的几何参数，通过查找上面提到的2D LUT，求解最终结果 */
    float sum = getSphereFormFactor(sinSigma, cosOmega);

    return sum;
}
```

# 16位浮点数的数值精度

让我们再次回顾多边形几何在Lambert的积分的向量形式，这次将重点放在其中一条边的积分上。
$$
\frac {acos(\langle p_i, p_j \rangle)}  {\| {p_i \times p_j} \|} {p_i \times p_j}
$$

对于给定的两个向量$p_i$与$p_j$，我们很容易通过$cos$与$cross$这两个函数对上面的表达式求解。

```
float3 integrateEdgeVectorForm(float3 p_i, float3 p_j)
{
    float cosTheta = dot(p_i, p_j)
    float sinTheta = sqrt(1 - cosTheta*cosTheta)
    float theta = acos(cosTheta)
    return cross(p_i, p_j) * theta / sinTheta;
}
```

上面的这段代码中包含了一个$acos$函数，以及$\frac {theta} {sinTheta}$，这些都值得我们注意，首先acos的开销较大，其次sinTheta可能接近0从而引起inf/nan，最后我们希望能够尽量提高FP16运算在整个计算过程中的占比。  

让我们以$t=cosTheta$重新参数化$\frac {theta} {sinTheta}$将得到$\frac {arccos(t)} {\sqrt {1-t^2}}$。

![](images/theta_div_sinTheta.png)

其中分子$arccos(t)$是关于$(0,\frac {\pi} {2})$的对称函数，$\frac 1 {\sqrt {1-t^2}}$为偶函数，所以定义域为[-1,0]的值可以通过定义域为[0,1]的结果计算得出，我们可以把注意力集中在[0,1]的范围内。

![](images/theta_div_sinTheta_01.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def f(cos_theta):
    sin_theta = np.sqrt(1 - cos_theta**2)
    theta = np.arccos(cos_theta)
    return theta / sin_theta

x = np.arange(-0.99,0.99,0.01)
y = f(x)
plt.figure(figsize=[16,8])
plt.plot(x, y, label=r"$\frac {arccos(t)} {\sqrt {1-t^2}}$")
plt.legend()
plt.xlim(-1,1)


In [ ]:
x = np.arange(0,1,0.01)
y = f(x)
plt.figure(figsize=[16,8])
plt.plot(x, y, label=r"$\frac {arccos(t)} {\sqrt {1-t^2}}$")
plt.legend()
plt.xlim(0,1)

我们使用二元多项式来拟合上述函数，最终得到$0.33735186 * x^2 - 0.89665001 * x + \frac {\pi} 2$这样一个多项式，相对错误率小于1%，整个多项式计算可以在16位浮点数下进行，满足我们对于移动平台的精度要求。

![](images/sin_div_sinTheta_polynomial.png)

In [ ]:
import scipy.optimize as optimize

def h(cos_theta):
    theta = np.arccos(cos_theta)
    sin_theta = np.sqrt(1-cos_theta**2)
    return theta / sin_theta

def approx(x, params):
    x_ = np.abs(x)
    v = x_**2 * params[0] + x_ * params[1] + np.pi/2
    v2 = np.pi/np.sqrt(1-x**2) - v
    return np.where(x>=0, v, v2)

def approx_fp16(x, params):
    x = x.astype(np.float16)
#     params = params.astype(np.float16)
    
    x_ = np.abs(x)
    v = x_**2 * params[0] + x_ * params[1] + np.pi/2
    v2 = np.pi/np.sqrt(1-x**2) - v
    ret = np.where(x>=0, v, v2)
    return ret

def obj_fn(params):
    x = np.arange(0,0.99, 0.001)
    y = h(x)
    y_ = approx(x, params)
    return np.sum((y - y_.astype(np.float64))**2)

initial_guess = [1,1,1,1,1,1]
ret = optimize.minimize(obj_fn, initial_guess)

x = np.arange(-0.99,0.99,0.01)
y = h(x)
y_ = approx(x, ret.x)
y_16 = approx_fp16(x, ret.x)
plt.figure(figsize=[16,8])
plt.plot(x, y, label="Target")
plt.plot(x, y_, label="Approximation FP32")
plt.plot(x, y_16, label="Approximation FP16")
plt.legend()
plt.xlim(-1,1)
print(ret.x)

In [ ]:
diff = np.abs(y_ - y)/y
diff_ = np.abs(y_16 - y)/y
plt.figure(figsize=[16,8])
plt.plot(x, diff, label="Error FP32")
plt.plot(x, diff_, label="Error FP16")
plt.legend()

# 菲涅尔项

在前面的实现中，我们并没有去考虑BRDF中的F项，即菲涅尔项。下面介绍一个将菲涅尔一起考虑的近似解。  
在半球面的积分恒等于1，是LTC的一个性质，然后因为BRDF中的G(遮挡项)，在不考虑菲涅尔的情况下，BRDF在半球面的积分小于1，所以需要存储一个单独的归一化系数为
$$
\int_\Omega D(\omega_h) G(\omega_i, \omega_o) cos(\theta_i) d\omega_i
$$

我们可以将菲涅尔项代入这个归一化系数得到将菲涅尔项一起考虑的近似解。
$$
\int_\Omega F(\omega_i, \omega_h) D(\omega_h) G(\omega_i, \omega_o) cos(\theta_i) d\omega_i
$$

细心的读者应该发现了，这样一个积分项与我们在Split sum approximation使用的BRDF预积分项一致，从而可以使用相同的打表以及纹理查询结果。

# 总结

本文做为对论文Real-time polygonal-light shading with linearly transformed cosines的补充，将原论文中由于篇幅而精简掉的理论部分做了完善，让读者对这个方法有更直观的理解。针对移动平台的性能要求，在性能与精度上做了一些取舍，在将水平面裁切通过查表来实现与使用多项式拟合之后，最终生成的指令数减少接近50%。

最终结果                          | 蒙特卡罗求解
------------------------------------|-------------------------------------
![](images/a.png)  | ![](images/b.png) 

# 引用


1. Heitz, E., Dupuy, J., Hill, S. & Neubelt, D. Real-time polygonal-light shading with linearly transformed cosines. Acm Transactions Graph Tog 35, 1–8 (2016).

2. Heitz, E. Analytical calculation of the solid angle subtended by an arbitrarily positioned ellipsoid to a point source. Nucl Instruments Methods Phys Res Sect Accel Spectrometers Detect Assoc Equip 852, 10–14 (2017).


3. Heitz, E. Geometric Derivation of the Irradiance of Polygonal Lights.
  

1.Iwanicki, M. l. Deriving the analytical formula for a diffuse response from a polygonal light source.


1.Hill, S. & Heitz, E. Real-Time Area Lighting: a Journey from Research to Production.
  
  
3. Snyder, J. M. Area Light Sources for Real-Time Graphics.
  